In [ ]:
import dualcodec
import torchaudio
from IPython.display import Audio

In [ ]:
model_id = "12hz_v1"

In [ ]:
path_1 = "output_checkpoints_2/dualcodec_experiments_fully_causal/checkpoint/epoch-0000_step-0094400_loss-135.731720-dualcodec_experiments_fully_causal"

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
path_2 = "output_checkpoints_2/dualcodec_experiments_fully_causal/checkpoint/epoch-0000_step-0083400_loss-169.211517-dualcodec_experiments_fully_causal"

dualcodec_model_2 = dualcodec.get_model(model_id, path_2)

In [ ]:
dualcodec_inference_1 = dualcodec.Inference(dualcodec_model=dualcodec_model_2, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference_1.encode(audio, n_quantizers=8)
out_audio = dualcodec_inference_1.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)